## 题目一：PaddleX GUI 图像分类任务结果示例

![](https://ai-studio-static-online.cdn.bcebos.com/ec49736e40304fde9f1d4bebd4c318837cd06920243642e9909a74633097ce64)


## 题目二：泪河分割任务

### 2.1 安装PaddleX

In [4]:
!pip install paddlex

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 522kB 17.2MB/s eta 0:00:01
     |████████████████████████████████| 102kB 62.0MB/s ta 0:00:01
     |████████████████████████████████| 153kB 26.3MB/s eta 0:00:01
     |████████████████████████████████| 1.0MB 12.0MB/s eta 0:00:01
  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl size=278345 sha256=d879347d4dbdb9ff7c1a6745c04371087481bc80daaabc393ad99e2cb33da7c5
  Stored in directory: /home/aistudio/.cache/pip/wheels/fb/44/67/8baa69040569b1edbd7776ec6f82c387663e724908aaa60963
Successfully built pycocotools


### 2.2 解压数据集

In [3]:
!unzip -oq data/json.zip
!unzip -oq data/jpeg.zip

unzip:  cannot find or open data/json.zip, data/json.zip.zip or data/json.zip.ZIP.
unzip:  cannot find or open data/jpeg.zip, data/jpeg.zip.zip or data/jpeg.zip.ZIP.


In [5]:
import matplotlib
matplotlib.use('Agg') 
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import paddlex as pdx

### 2.3、数据集格式转换

In [11]:
!paddlex --data_conversion --source labelme --to SEG \
        --pics  jpeg \
        --annotations json \
        --save_dir dataset_seg

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


### 2.4、划分数据集

In [12]:
!paddlex --split_dataset --format SEG --dataset_dir dataset_seg --val_value 0.2 --test_value 0.1

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
Dataset Split Done.
Train samples: 293
Eval samples: 83
Test samples: 41
Split files saved in dataset_seg


### 2.5 数据预处理

In [13]:
from paddlex.seg import transforms
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Resize(target_size=512),
    transforms.RandomPaddingCrop(crop_size=500),
    transforms.Normalize()
])
eval_transforms = transforms.Compose([
    transforms.Resize(512),
    transforms.Normalize()
])

### 2.6 训练参数配置

In [14]:
train_dataset = pdx.datasets.SegDataset(
    data_dir='dataset_seg',
    file_list='dataset_seg/train_list.txt',
    label_list='dataset_seg/labels.txt',
    transforms=train_transforms,
    shuffle=True)
eval_dataset = pdx.datasets.SegDataset(
    data_dir='dataset_seg',
    file_list='dataset_seg/val_list.txt',
    label_list='dataset_seg/labels.txt',
    transforms=eval_transforms)

2021-04-21 19:16:53 [INFO]	293 samples in file dataset_seg/train_list.txt
2021-04-21 19:16:53 [INFO]	83 samples in file dataset_seg/val_list.txt


### 2.7、模型训练

In [15]:
num_classes = len(train_dataset.labels)
model = pdx.seg.DeepLabv3p(num_classes=num_classes)
model.train(
    num_epochs=40,
    train_dataset=train_dataset,
    train_batch_size=4,
    eval_dataset=eval_dataset,
    learning_rate=0.01,
    save_interval_epochs=1,
    save_dir='output/deeplab',
    use_vdl=True)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:298: UserWarning: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlex/cv/nets/segmentation/model_utils/loss.py:74
The behavior of expression A * B has been unified with elementwise_mul(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elementwise_mul(X, Y, axis=0) instead of A * B. This transitional warning will be dropped in the future.
  op_type, op_type, EXPRESSION_MAP[method_name]))
2021-04-21 19:16:57,498 - INFO - If regularizer of a Parameter has been set by 'fluid.ParamAttr' or 'fluid.WeightNormParamAttr' already. The Regularization[L2Decay, regularization_coeff=0.000040] in Optimizer will not take effect, and it will only be applied to other Parameters!


2021-04-21 19:16:58 [INFO]	Downloading MobileNetV2_pretrained.tar from https://paddle-imagenet-models-name.bj.bcebos.com/MobileNetV2_pretrained.tar


100%|██████████| 14060/14060 [00:00<00:00, 24778.60KB/s]


2021-04-21 19:16:59 [INFO]	Decompressing output/deeplab/pretrain/MobileNetV2_pretrained.tar...
2021-04-21 19:17:04 [INFO]	Load pretrain weights from output/deeplab/pretrain/MobileNetV2_pretrained.
2021-04-21 19:17:04 [INFO]	There are 260 varaibles in output/deeplab/pretrain/MobileNetV2_pretrained are loaded.
2021-04-21 19:17:11 [INFO]	[TRAIN] Epoch=1/40, Step=2/73, loss=0.526934, lr=0.009997, time_each_step=3.34s, eta=3:29:16
2021-04-21 19:17:12 [INFO]	[TRAIN] Epoch=1/40, Step=4/73, loss=0.198357, lr=0.009991, time_each_step=1.78s, eta=1:51:13
2021-04-21 19:17:12 [INFO]	[TRAIN] Epoch=1/40, Step=6/73, loss=0.097528, lr=0.009985, time_each_step=1.28s, eta=1:20:17
2021-04-21 19:17:13 [INFO]	[TRAIN] Epoch=1/40, Step=8/73, loss=0.062337, lr=0.009978, time_each_step=1.04s, eta=1:4:47
2021-04-21 19:17:13 [INFO]	[TRAIN] Epoch=1/40, Step=10/73, loss=0.06064, lr=0.009972, time_each_step=0.87s, eta=0:54:21
2021-04-21 19:17:14 [INFO]	[TRAIN] Epoch=1/40, Step=12/73, loss=0.082443, lr=0.009966, time

100%|██████████| 21/21 [00:08<00:00,  2.39it/s]


...

2021-04-21 19:32:06 [INFO]	[EVAL] Finished, Epoch=38, miou=0.880972, category_iou=[0.99730991 0.76463363], oacc=0.997333, category_acc=[0.99876725 0.85692458], kappa=0.865274, category_F1-score=[0.99865314 0.86662026] .
2021-04-21 19:32:06 [INFO]	Model saved in output/deeplab/epoch_38.
2021-04-21 19:32:06 [INFO]	Current evaluated best model in eval_dataset is epoch_34, miou=0.8822348606711969
2021-04-21 19:32:10 [INFO]	[TRAIN] Epoch=39/40, Step=2/73, loss=0.010572, lr=0.00067, time_each_step=0.26s, eta=0:0:54
2021-04-21 19:32:11 [INFO]	[TRAIN] Epoch=39/40, Step=4/73, loss=0.004513, lr=0.000662, time_each_step=0.28s, eta=0:0:56
2021-04-21 19:32:12 [INFO]	[TRAIN] Epoch=39/40, Step=6/73, loss=0.013239, lr=0.000654, time_each_step=0.3s, eta=0:0:57
2021-04-21 19:32:12 [INFO]	[TRAIN] Epoch=39/40, Step=8/73, loss=0.00463, lr=0.000645, time_each_step=0.32s, eta=0:0:57
2021-04-21 19:32:12 [INFO]	[TRAIN] Epoch=39/40, Step=10/73, loss=0.005689, lr=0.000637, time_each_step=0.33s, eta=0:0:57
2021-0

100%|██████████| 21/21 [00:12<00:00,  1.72it/s]


2021-04-21 19:32:30 [INFO]	[EVAL] Finished, Epoch=39, miou=0.880656, category_iou=[0.99729085 0.76402063], oacc=0.997314, category_acc=[0.99879894 0.85313921], kappa=0.86487, category_F1-score=[0.99864359 0.86622641] .
2021-04-21 19:32:31 [INFO]	Model saved in output/deeplab/epoch_39.
2021-04-21 19:32:31 [INFO]	Current evaluated best model in eval_dataset is epoch_34, miou=0.8822348606711969
2021-04-21 19:32:34 [INFO]	[TRAIN] Epoch=40/40, Step=1/73, loss=0.006039, lr=0.000362, time_each_step=0.24s, eta=0:0:29
2021-04-21 19:32:35 [INFO]	[TRAIN] Epoch=40/40, Step=3/73, loss=0.023448, lr=0.000353, time_each_step=0.26s, eta=0:0:30
2021-04-21 19:32:35 [INFO]	[TRAIN] Epoch=40/40, Step=5/73, loss=0.00686, lr=0.000344, time_each_step=0.28s, eta=0:0:31
2021-04-21 19:32:36 [INFO]	[TRAIN] Epoch=40/40, Step=7/73, loss=0.00712, lr=0.000335, time_each_step=0.3s, eta=0:0:32
2021-04-21 19:32:36 [INFO]	[TRAIN] Epoch=40/40, Step=9/73, loss=0.005263, lr=0.000326, time_each_step=0.32s, eta=0:0:32
2021-04-

100%|██████████| 21/21 [00:09<00:00,  2.16it/s]


2021-04-21 19:32:52 [INFO]	[EVAL] Finished, Epoch=40, miou=0.88174, category_iou=[0.99734068 0.76613857], oacc=0.997364, category_acc=[0.99874029 0.86143351], kappa=0.866255, category_F1-score=[0.99866857 0.86758603] .
2021-04-21 19:32:52 [INFO]	Model saved in output/deeplab/epoch_40.
2021-04-21 19:32:52 [INFO]	Current evaluated best model in eval_dataset is epoch_34, miou=0.8822348606711969


### 2.8 模型预测

In [9]:
model = pdx.load_model('output/deeplab/best_model')
image_name = 'dataset_seg/JPEGImages/lhgd227.jpg'
result = model.predict(image_name)
pdx.seg.visualize(image_name, result, weight=0.4, save_dir='./output/deeplab')

2021-04-21 22:39:16 [INFO]	Model[DeepLabv3p] loaded.
2021-04-21 22:39:16 [INFO]	The visualized result is saved as ./output/deeplab/visualize_lhgd227.jpg


#### 模型预测效果展示

![](https://ai-studio-static-online.cdn.bcebos.com/59bcb2b7113f421fbfda27c429d94bdab91db1fe6d2b4e909ce98121c9927bf1)
